<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="https://raw.githubusercontent.com/holoviz/holoviz/master/doc/_static/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 5. Real-time dashboard</h2></div>

- Can you write a Python function that returns a Pandas dataframe?
- Once you have a Pandas dataframe, can you call ".plot()" on it?

If so, congratulations; you can write a streaming interactive app with dynamically updating visualizations!

This example shows you how to configure the Python streamz library for the special (and easy!) case of periodically calling a Python function to get some data in the form of a Pandas dataframe. The resulting "streaming dataframe" automatically updates as new data is available, without any additional work from you.

Your streaming dataframe can then be visualized using the standard Pandas plotting API, which usually returns a static image but when used with hvPlot will give you fully dynamic, reactive plots.

For this example, we will query the system's memory usage to make a little Memory monitoring app. 

## Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot streamz`.

In [ ]:
# !pip install panel==0.12.6 hvplot==0.7.3 streamz

In [ ]:
import pandas as pd, time, psutil
from streamz.dataframe import PeriodicDataFrame
import hvplot.streamz, holoviews as hv
import panel as pn

## Get data

First, we make a function `mem_data` to get memory data for our system. The output is a pandas dataframe with index representing the current timestamp.

In [ ]:
def mem_data(**kwargs):
      return pd.DataFrame(
          dict(psutil.virtual_memory()._asdict()),
          index=[pd.Timestamp.now()]
      )

In [ ]:
mem_data()

Second, we use `streamz` to create a streaming dataframe based on the memory usage data. The function streaming_weather_data is used as a callback function by the PeriodicDataFrame function to create a streamzstreaming dataframe df. streamzdocs documented how PeriodicDataFrame works:

"streamz provides a high-level convenience class for this purpose, called a PeriodicDataFrame. A PeriodicDataFrame uses Python’s asyncio event loop (used as part of Tornado in Jupyter and other interactive frameworks) to call a user-provided function at a regular interval, collecting the results and making them available for later processing."

https://streamz.readthedocs.io/en/latest/dataframes.html#streaming-dataframes

In [ ]:
df = PeriodicDataFrame(mem_data, interval='300ms')


In [ ]:
df

## Plotting 

The above dataframes should periodically update every 0.3 seconds with the results of calling mem_data, always reporting the most current data.

We can now use df just about anywhere we'd use a regular Pandas DataFrame. In particular, we can very easily set up a little graphical app to monitor CPU and memory usage, updating every 300ms, using hvPlot:

In [ ]:
df[['used']].hvplot.line(title="Memory Usage", backlog=200)

In [ ]:
df[['free']].hvplot.line(title="Memory Usage", backlog=200)

Here we used `.hvplot()` to get hvPlot-based dynamic plots instead of static images, each driven by the streamz dataframes so that their data always updates (showing up to the most recent 200 data points). See hvplot.holoviz.org for all the types of plots you can use, and holoviews.org for the Bokeh-based options like cmap that you can tweak if you want and for other layout options. This code only gets one value per callback invocation, but see the streamz docs if you need better throughput that you can get from querying data in bigger chunks, e.g. from a file or by accessing a database.

## Dashboarding

What if we would like to make a dashboard that allows us to choose which value we would like to see in a plot? 

First, let's make a Panel widget to select the column name:

In [ ]:
select_column = pn.widgets.Select(options=df.columns.to_list(), name='Column')

In [ ]:
select_column

Second, we make a plotting function with the parameter representing the column selected:

In [ ]:
def select_plot(select_column):
    return df[[select_column]].hvplot.line(
        title="Memory Usage", 
        backlog=200,
        line_width=6, 
        height=400,
        color="#ff6f69"
    )

In [ ]:
select_plot("total")

Finally, we can create a dashboard. Since our df is a streamz dataframe, the `.interactive` function we used in the previous tutorial doesn't not work with the streamz dataframe. Instead, a second way of creating dashboard is to use `pn.bind` to bind our Panel widget and the plot.


"The `pn.bind` reactive programming API is very similar to the `interact` function but is more explicit about widget selection and layout. pn.bind requires the programmer to select and configure widgets explicity and to lay out components explicitly, without relying on inference of widget types and ranges and without any default layouts. Specifying those aspects explicitly provides more power and control, but does typically take a bit more code and more knowledge of widget and layout components than using interact does. Once widgets have been bound to a reactive function, you can lay out the bound function and the widgets in any order or combination you like, including across Jupyter notebook cells if desired. (https://panel.holoviz.org/user_guide/APIs.html) " 

In [ ]:
dash = pn.Column(
    select_column, 
    pn.bind(select_plot, select_column=select_column)
)
dash

To *serve the notebook* run `panel serve 05_Real_Time_Dashboard.ipynb`.

In [ ]:
# dash.show()
dash.servable();